# Applying Welch's t-test on ASCAD

In [1]:
from tqdm import tqdm
from src.data.slices import tvla_slice
from src.data.smote import smote
from src.data.traceloader import AscadRandomKey
import numpy as np
from src.tvla.main import tvla
from src.tvla.t import make_t_test

In [2]:
ascad = AscadRandomKey()

In [3]:
ascad.default.profile.filter_by_hw(False).shape, ascad.default.profile.filter_by_key(0).shape

((199233, 1400), (760, 1400))

In [4]:
P_VALUE = 5
PT_OF_INTEREST = (50, 80)

def prepare(make_test, samples, p=P_VALUE, poi=PT_OF_INTEREST):
    slc = [make_slice(samples.filter_by_key(i)) for i in range(ascad.key_size)]
    max_slice_size = max([len(s[0]) for s in slc])

    slc_smote = np.array([smote_slices(s, max_slice_size) for s in slc])

    slc_mv = np.moveaxis(np.array((slc_smote.mean(axis=2), slc_smote.var(axis=2))), 0, -1)

    test = make_test(max_slice_size)

    return test, slc_mv[:,:,poi[0]:poi[1]]

# masked = prepare(make_t_test, ascad.masked.profile)

default = prepare(make_t_test, ascad.default.profile)
random = prepare(make_t_test, ascad.random.profile)

/home/berg/Documents/Afstudeer/LA/src/data/slices.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([traces[s] for s in np.array_split(select, 4)])


In [5]:
tvla(test=default[0], left=default[1][0], right=default[1][208], p=0.01)

(True, False)

In [6]:
tvla(test=random[0], left=random[1][0], right=random[1][208], p=0.01)

(False, True)

In [7]:
def key_tvla(test, slices: np.array, key_size=ascad.key_size):
    res = np.array(np.zeros((key_size, key_size)), dtype=int)
    progress = tqdm(total=key_size)
    ff, tf, ft, tt = 0, 0, 0, 0

    for i in range(key_size):
        a = slices[i]
        for j in range(key_size):
            b = slices[j]

            if i != j:
                la = tvla(test=test, left=a, right=b, p=5)
                enc = 2 * la[0] + la[1]
                res[i][j] = enc

                ff += enc == 0
                ft += enc == 1
                tf += enc == 2
                tt += enc == 3

        progress.set_description(f"Total ({[ff, ft, tf, tt]})")
        progress.update()

    progress.close()
    return res


res_default = key_tvla(*default)
res_random = key_tvla(*random)

Total ([0, 12980, 14121, 38179]): 100%|██████████| 256/256 [00:41<00:00,  6.15it/s]


In [8]:
res_default.sum()



156716